<a href="https://colab.research.google.com/github/MateusFelipes/Atualizador-de-Clientes/blob/main/ADICIONAR_CLIENTES.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# === Limpeza inicial dos arquivos no kernel ===
import os, glob
for f in glob.glob('*'):
    try:
        os.remove(f)
    except IsADirectoryError:
        pass

# === 1) Upload do arquivo de CLIENTES ===
from google.colab import files
import sys

print("Faça upload do arquivo CLIENTES:")
uploaded = files.upload()
if len(uploaded) != 1:
    print("❌ Envie exatamente 1 arquivo de clientes.")
    sys.exit(1)
clientes_filename = list(uploaded.keys())[0]
print(f"Arquivo de clientes recebido: {clientes_filename}")

# === 2) Upload do arquivo do HUBSPOT ===
print("\nFaça upload do arquivo do HUBSPOT:")
uploaded = files.upload()
if len(uploaded) != 1:
    print("❌ Envie exatamente 1 arquivo do hubspot.")
    sys.exit(1)
hubspot_filename = list(uploaded.keys())[0]
print(f"Arquivo do hubspot recebido: {hubspot_filename}")

# === 3) Parâmetros de aba e coluna ===
clientes_sheet   = input("\nNome da aba NO CLIENTES (ex: 'Cliente Ativo'): ")
hubspot_sheet    = input("Nome da aba NO HUBSPOT (ex: 'Cliente Ativo'): ")
col_name_clientes = input("\nNome da coluna NO CLIENTES que guarda o nome do cliente (ex: 'Nome da empresa'): ")
col_name_hubspot  = input("Nome da coluna NO HUBSPOT que guarda o nome do cliente (ex: 'Company name'): ")

# === 4) Importa libs e lê dados ===
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils.cell import coordinate_from_string

clientes_df = pd.read_excel(clientes_filename, sheet_name=clientes_sheet, usecols=[col_name_clientes])
hubspot_df  = pd.read_excel(hubspot_filename,  sheet_name=hubspot_sheet,  usecols=[col_name_hubspot])

# === 5) Filtra novos clientes ===
novos = hubspot_df[~hubspot_df[col_name_hubspot].isin(clientes_df[col_name_clientes])]
if novos.empty:
    print("\n✅ Não há novos clientes a adicionar.")
    # limpa uploads antes de sair
    os.remove(clientes_filename)
    os.remove(hubspot_filename)
    sys.exit(0)

# === 6) Abre workbook, encontra tabela e expande intervalo ===
wb = load_workbook(clientes_filename)
ws = wb[clientes_sheet]

table_names = list(ws.tables.keys())
if not table_names:
    raise ValueError("❌ Nenhuma Tabela oficial do Excel encontrada nesta aba.")
table_name = table_names[0]
tbl = ws.tables[table_name]

start_cell, end_cell = tbl.ref.split(':')
start_col, start_row = coordinate_from_string(start_cell)
end_col,   end_row   = coordinate_from_string(end_cell)

new_end_row = end_row + len(novos)
tbl.ref = f"{start_col}{start_row}:{end_col}{new_end_row}"

# === 7) Insere linhas na tabela e preenche nomes ===
insert_at = end_row + 1
ws.insert_rows(insert_at, amount=len(novos))

header = {cell.value: cell.column for cell in ws[1]}
if col_name_clientes not in header:
    raise ValueError(f"❌ Coluna '{col_name_clientes}' não encontrada no cabeçalho.")
col_idx = header[col_name_clientes]

for i, nome in enumerate(novos[col_name_hubspot], start=insert_at):
    ws.cell(row=i, column=col_idx, value=nome)

# === 8) Salva e dispara download ===
out_name = 'clientes_atualizado.xlsx'
wb.save(out_name)
print(f"\n✅ {len(novos)} cliente(s) inserido(s) e Tabela expandida com sucesso em '{out_name}'.")
files.download(out_name)

# === 9) Limpa arquivos carregados ===
os.remove(clientes_filename)
os.remove(hubspot_filename)


Faça upload do arquivo CLIENTES:


Saving clientes.xlsx to clientes.xlsx
Arquivo de clientes recebido: clientes.xlsx

Faça upload do arquivo HUBSPOT:


Saving hubspot-crm-exports-cliente-ativo-2025-05-06.xlsx to hubspot-crm-exports-cliente-ativo-2025-05-06.xlsx
Arquivo do hubspot recebido: hubspot-crm-exports-cliente-ativo-2025-05-06.xlsx

Nome da aba NO CLIENTES (ex: 'Cliente Ativo'): Cliente Ativo
Nome da aba NO HUBSPOT (ex: 'Cliente Ativo'): Cliente Ativo

Nome da coluna NO CLIENTES que guarda o nome do cliente (ex: 'Nome da empresa'): Nome da empresa
Nome da coluna NO HUBSPOT que guarda o nome do cliente (ex: 'Company name'): Company name

✅ 20 cliente(s) inserido(s) e Tabela expandida com sucesso em 'clientes_atualizado.xlsx'.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>